_[Please follow the instructions [here](https://github.com/Hermi999/bce) to see how to properly use this bash-kernel Notebook]_

# Beyond Corp - Walkthrough

The following notebook is used to guide you through setting up Beyond Corp on Google Cloud Platform (GCP). 

## Context
As more employees work from home, it is important to provide more options for them to securely access internal services and sites.
These sites may be your intranet portal, sensitive HR or financial systems that are not exposed directly externally. Traditionally, companies accessed these sites after establishing a VPN tunnel from an employee’s laptop directly to the corporate network. Just about any administrator of such a system will tell you VPNs are expensive to procure, require a lot of resources, quite cumbersome to maintain and essentially provide security based on dynamically expanding the trusted network.

At Google, we found the access model based primarily on a network perimeter did not sufficiently scale with a mobile workforce nor provide selective security guarantees for sensitive applications. Our approach was to assess the overall context of a given request for an application. By that we would take signals derived from multiple sources at runtime to evaluate the integrity of a call against the resource being accessed. For example, the evaluation signals for a request would include who is making the call, from what device, from what network, at what time, and so on. A request is not simply evaluated using an administrator defined static if-then chain, but also assigned trust levels based on heuristics inferred from prior contexts.

The internal system Google used to achieve this is described in six BeyondCorp research papers which goes into more details about these signals. For our customers we make this same system available as __BeyondCorp (BCE).__

As you can see on the image below, BCE consists of multiple components:
- Cloud Identity
- Identity Aware proxy
- Google Cloud Load Balancer
- Access Context Manager
- IAP Connector

![BCE](./img/bce.png)

## How to use this colab notebook

*   If you didn't start at following Github Repository, please start there to see how you can use this bash-kernel based notebook: https://github.com/Hermi999/bce
*   **You can use this notebook either as a reference and copy the commands directly into GCP Cloud Shell, or you use it with a Jupyter Notebook with a bash kernel.**
*   If you like to execute a block use STRG+ENTER
*   If you just want to run highlighted commands use STRG+SHIFT+ENTER

## Getting ready
Let's try it out by checking the version of the gcloud sdk. Click into the next line and press STRG+ENTER

In [ ]:
gcloud version

We do not need to authenticate ourselfs to run gcloud commands. It automatically uses the Compute Engine Service Account which was copied onto the VM during creation.

You can check this by running the command:

In [ ]:
gcloud auth list

This Service Account should have all necessary rights to proceed.

We also double check if we are in the right project:

In [ ]:
gcloud projects list

If the project is wrong you can set your project with 'gcloud config set project [project_name].

## Deploy a first sample application
We start with a first sample application on GCP. 
For this we will use the __App Engine__, which was the first GCP service (2008) and is a PaaS for hosting web applications. 

Our Git repository we cloned onto this machine contains an App Engine sample code. 

In [ ]:
cd ~/bce/samples/appengine
ls

Now we can deploy the simple sample to App Engine:

In [ ]:
gcloud app create --region=europe-west3

Wait until you see the message gcloud app browse

In [ ]:
gcloud app deploy --quiet

In [ ]:
gcloud app browse

When you follow the link you should see an empty website with a simple sign in link.
Now that we have our app running it's time to secure it with Identity-aware proxy.

## Configure OAuth 
### The Consent Screen (=Brand)
(https://cloud.google.com/iap/docs/programmatic-oauth-clients)

Because IAP accesses Google users’ data (email address) you need to configure the OAuth.
This needs to be done once per GCP Project. Typically you manage multiple on-prem or AWS/Azure web apps from the same projects, so it only need to be done rearily. 

The OAuth consent screen, which contains branding information for users, is known as a brand. Brands can be limited to internal users or public users. An __internal brand__ makes the OAuth flow accessible to someone who belongs to the same Google Workspace organization as the project. A __public brand__ makes the OAuth flow available to anyone on the internet. There is only one brand per GCP project.

OAuth API verification is a little bit difficult to understand - read more about it here: https://support.google.com/cloud/answer/9110914

Let's create our brand:

In [ ]:
APP_TITLE=IAP_TEST
SUPPORT_EMAIL=[enter your email]

gcloud alpha iap oauth-brands create --application_title=$APP_TITLE --support_email=$SUPPORT_EMAIL

The support email displayed on the OAuth consent screen. This email address can either be a user's address or a Google Groups alias.

Now we can list our new brand and extract the Brand-ID:

In [ ]:
gcloud alpha iap oauth-brands list
BRAND=$(gcloud alpha iap oauth-brands list --format="value(name)")

echo "BRAND: $BRAND"

### The OAuth Client
(https://cloud.google.com/iap/docs/programmatic-oauth-clients#creating_an_oauth_client)

Now we create our OAuth Client for the IAP. You configure one OAuth client per app.

In [ ]:
CLIENT_NAME=iap
PROJECT_ID=$(gcloud config get-value core/project)

gcloud alpha iap oauth-clients create $BRAND --display_name=$CLIENT_NAME
#gcloud alpha iap oauth-clients create projects/$PROJECT_ID/brands/1023990209690 --display_name=NAME

Be aware that it's only possible to create oauth clients via gcloud if the brand is __internal__. Otherwise you have to use the GUI.

Now we enable IAP for the web service. Fill in the client_id and secret from the previous step:

In [ ]:
CLIENT_ID=
SECRET=
gcloud alpha iap web enable --resource-type=app-engine --oauth2-client-id=$CLIENT_ID --oauth2-client-secret=$SECRET

Now we have activated IAP for the App Engine App. This means that we do not have access any more...

In [ ]:
gcloud app browse

## Creating access levels
https://cloud.google.com/access-context-manager/docs/overview

While Identity-Aware Proxy (IAP) lets you manage who has access to your web services, with __Access Context Manager__ you can define __when__ (in which circumstances) the request is allowed. Access Context Manager allows Google Cloud organization administrators to define fine-grained, attribute based access control for projects and resources in Google Cloud.

Administrators first define an __access policy__, which is an organization-wide container for __access levels__.
Access levels describe the necessary requirements for requests to be honored. Examples include:
- Device type and operating system
- IP address
- User identity
- Device Type (you need a BeyondCorp or Cloud Identity Premium license)

### Use GitOps for Access Levels
We could create access levels via the GCP Console UI, but that's too easy :) Instead we want to set up a GitOps-based approach for our access levels. This makes it easy to track changes and approve changes which can be submitted via a __merge request__.

#### Create the accss policy
First we can [create an access policy](https://cloud.google.com/access-context-manager/docs/create-access-policy) for __Access Context Manager__. 1 Organization can only have 1 access policy:

In [ ]:
POLICY_TITLE="my_org_policy"
PROJECT_ID=$(gcloud config get-value core/project)
ORG_ID=$(gcloud projects describe $PROJECT_ID --format="value(parent.id)")

gcloud access-context-manager policies create --organization $ORG_ID --title $POLICY_TITLE

#### Define Access levels

https://cloud.google.com/access-context-manager/docs/create-basic-access-level

Now let's have a look at the example access context manager __access level policies__ I've prepared for you. For this exercise I just created 2 simple policies:

__Low Trust (lowtrust.yaml):__
Simply checks if the users accesses the app from one of the following 3 countries: Austria, Italy or Germany.

__High Trust (hightrust.yaml):__
Build upon the Low Trust policy, but additionally limits access to the CIDR: 80.110.0.0/16

In [48]:
cat ~/bce/accesscontextmngr/lowtrust.yaml

- ipSubnetworks:
  - 0.0.0.0/0
- regions:
  - IT
  - DE
  - AT


In [49]:
cat ~/bce/accesscontextmngr/hightrust.yaml

- ipSubnetworks:
  - 80.110.0.0/16
  requiredAccessLevels:
  - accessPolicies/POLICY_ID/accessLevels/Low_Trust



I also included a more complex example which includes specific device policies. We will not use this for our exercise here because you need to have a BeyondCorp license or Cloud Identity Premium. Still let's have a closer look at it:

In [ ]:
cat ~/bce/accesscontextmngr/complex.yaml

----------------------------------------------------------------
As you can see, the access level policies are easy to create, read and can also be combined.

We could now simply apply them with the following gcloud command:
````bash
gcloud access-context-manager levels create ...
````

But we don't do this now because we want to set up a Gitops-based pipeline to automatically deploy new access levels whenever the admin commits them to a git repository....

If you still want to try already (because you can't wait) it out here are the commands:

In [50]:
ORG_ID=$(gcloud projects describe $PROJECT_ID --format="value(parent.id)")
POLICY_NAME=$(gcloud access-context-manager policies list --organization=$ORG_ID --format="value(NAME)")
cd ~/bce/accesscontextmngr
gcloud access-context-manager levels create "Low_Trust" --basic-level-spec=lowtrust.yaml --combine-function=OR --policy=$POLICY_NAME --title="Low Trust"
# replace POLICY_ID with actual ID of the policy
sed "s/POLICY_ID/$POLICY_NAME/g" hightrust.yaml > hightrust2.yaml
gcloud access-context-manager levels create "High_Trust" --basic-level-spec=hightrust2.yaml --combine-function=OR --policy=$POLICY_NAME --title="High Trust"

# UI
echo "https://console.cloud.google.com/security/access-level?organizationId=$ORG_ID"

Create request issued for: [Low_Trust]
Waiting for operation [operations/accessPolicies/1082987218928/accessLevels/Low
_Trust/create/1605400992768540] to complete...done.                            
Created level [Low_Trust].
Create request issued for: [High_Trust]
Waiting for operation [operations/accessPolicies/1082987218928/accessLevels/Hig
h_Trust/create/1605400998417291] to complete...done.                           
Created level [High_Trust].
https://console.cloud.google.com/security/access-level?organizationId=834642112511


In the UI it will look something like this:
![](img/accesscontextmanager.png)

We can also export all access level policies, change them and then re-import them again:

In [ ]:
# Export all
gcloud access-context-manager levels list --policy $POLICY_NAME --format=json > all.json

Now you can open and modify the file...

... And then re-import all access level policies:

In [ ]:
gcloud access-context-manager levels replace-all $POLICY_NAME --source-file=all.json

If you created the access policies, let's clean up again:

In [ ]:
gcloud access-context-manager levels delete "High_Trust" --policy=$POLICY_NAME -q
gcloud access-context-manager levels delete "Low_Trust" --policy=$POLICY_NAME -q

#### Prepare Cloud Build file
Now that we know how to deploy, update and delete access level policies, let's put those commands into a file so that we can use it for an automated deployment via a Gitops-based approach.

While it's pretty straight forward to use gcloud to create, update and delete level policies, we have to think on how we can elegantly resolve levels which depend on each other. When we look at the hightrust.yaml file, we can see that the reference to the lowtrust level contains the policy ID:

````bash
...
requiredAccessLevels:
  - accessPolicies/POLICY_ID/accessLevels/Low_Trust
````

Now we could hard-code the Policy ID into our yaml files, but it's better to dynamically replace it before deploying the yaml to the environment. This makes sure that we don't need to adapt all our policy files when we use for another policy ID.

Another problem is that the the level yaml spec files can't include the "name", "title", "description" and "combine function" and that we need to provide this information via the _gcloud access-context-manager levels create_ command. --> The question is how/where we can store this information in Git? 

For the moment (maybe there will be improvement of Context-aware access in future) it seems like we need to put this information into our yaml files and parse and remove this information before deploying with _gcloud access-context-manager levels create_. Interestingly, when we show an existing level policy with  __gcloud access-context-manager levels describe__ the level policy is already shown in a format containing the above mentioned attributes: 

````bash
basic:
  combiningFunction: OR
  conditions:
  - ipSubnetworks:
    - 0.0.0.0/0
  - regions:
    - IT
    - DE
    - AT
description: Low Trust Access Level
name: accessPolicies/1082987218928/accessLevels/Low_Trust
title: Low Trust
````

But atm we can't re-import an access level exported via the describe command (at least not that I know of). Still we can use exactly this format (maybe it's later possible to import directly using this format) to store our access levels and then render them into the format which is necessary at the moment + the corresponding gcloud command. The yaml above will turn into:

````bash
- ipSubnetworks:
  - 0.0.0.0/0
- regions:
  - IT
  - DE
  - AT
````

AND the following gcloud commands:

````bash
gcloud access-context-manager levels create accessPolicies/1082987218928/accessLevels/Low_Trust --basic-level-spec=lowtrust.yaml --combine-function=OR --title="Low Trust" --description="Low Trust access level"

gcloud access-context-manager levels update accessPolicies/1082987218928/accessLevels/Low_Trust --basic-level-spec=lowtrust.yaml --combine-function=OR --title="Low Trust" --description="Low Trust access level"

````

...

...

#### Create the Continuious Deployment
For simplicity we will use [Cloud Source Repositories](https://cloud.google.com/source-repositories) for setting up our Git-based deployment, but you can use any SCM tool you like (e.g. Bitbucket, Gitlab, Github, etc.).

We start with copying the Access Level Config YAML files to a new folder:

In [ ]:
cp -r ~/bce/accesscontextmngr ~

Now let's create a new Cloud Source Repository:

In [ ]:
ACM_NAME="acm_staging"
gcloud source repos create $ACM_NAME

Now we initialize the new Git Repo and push the code from the local Git repository to the Cloud Source Repository we created in the previous step. Replace the __email__ and __name__:

In [ ]:
YOUR_EMAIL="h@hewagner.com"
YOUR_NAME="Hermann Wagner"
cd ~/accesscontextmngr
git init
git config --global user.email $YOUR_EMAIL
git config --global user.name $YOUR_NAME
git add -A
git commit -am "initial commit"

In [ ]:
git config --global credential.https://source.developers.google.com.helper gcloud.sh
git remote add google https://source.developers.google.com/p/hewagner-demos-2/r/acm_staging
git push --all google

If you want you can have a look at it in the browser by opening the following link:

In [ ]:
PROJECT_ID=$(gcloud config get-value core/project)
echo "https://source.cloud.google.com/$PROJECT_ID/$ACM_NAME"

![](img/cloudsourcerepo.png)

What we want is that every time we do a commit to the master branch, our policies get automatically applied to the access context manager. 

For this we use [Cloud Build](https://cloud.google.com/cloud-build/) which get's triggered by a commit:

In [ ]:
gcloud beta builds triggers create cloud-source-repositories \
    --repo=$ACM_NAME \
    --branch-pattern="master" \
    --build-config=cloudbuild.yaml

Now that we have our trigger we can make a small change and commit the change. This will trigger a new __Cloud build__ which will deploy our access level policies to __Access Context Manager__. 

Before we trigger the build, let's open Cloud build in the UI to see how it's executing the build once we trigger it:

In [ ]:
PROJECT_ID=$(gcloud config get-value core/project)
echo "https://console.cloud.google.com/cloud-build/builds?project=$PROJECT_ID"

After we execute the next step a new entry will appear in the table. You can click on the link in the "Build" column.

In [ ]:
echo "new test">test.yaml
git commit -am "minor change"
git push --all google

Here is what you should see in the UI:
![](img/cloudbuildhistory.png)

When you follow the link you see the details of the build:
![](img/cloudbuilddetails.png)

## Grant access on the application
The last step is to grant access to the protected app. 
We can do this on an individual user or group level and grant access to just a single app or to all apps in the whole project. First let's add yourself:

In [ ]:
USER="h@hewagner.com"
gcloud iap web add-iam-policy-binding --member="user:$USER" --role='roles/iap.httpsResourceAccessor'

Wait about a minute and try to access the web app url:

In [ ]:
gcloud app browse

You should see a welcome message and your email address.

## GCP Console
Of course you can also create and view everything we just did in a GUI (GCP Console). It should look similar to this:
![alt](img/iap.png)

Here are the links to have a look for yourself:

In [ ]:
PROJECT_ID=$(gcloud config get-value core/project)
ORG_ID=$(gcloud projects describe $PROJECT_ID --format="value(parent.id)")
echo "https://console.cloud.google.com/security/iap?project=$PROJECT_ID"
echo "https://console.cloud.google.com/security/access-level?orgonly=true&organizationId=$ORG_ID"